# Game Viewer
Replay saved tournament games. Set the `GAME_PATH` below and run all cells.

In [ ]:
import chess
import chess.svg
import json
import os
import time
from IPython.display import display, SVG, clear_output

DATA_DIR = os.path.join(os.getcwd(), "data")

## Browse Sessions & Games

In [ ]:
sessions = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
for i, session in enumerate(sessions, 1):
    summary_path = os.path.join(DATA_DIR, session, "summary.json")
    if os.path.exists(summary_path):
        with open(summary_path) as f:
            s = json.load(f)
        print(f"{i}. {session}")
        print(f"   {s['bot1']} {s['bot1_wins']} - {s['draws']} - {s['bot2_wins']} {s['bot2']}")
    else:
        print(f"{i}. {session}")
        games = sorted([f for f in os.listdir(os.path.join(DATA_DIR, session)) if f.startswith("game_") and f.endswith(".json")])
        print(f"   {len(games)} game(s), no summary")

In [ ]:
# Pick a session number from above
SESSION_NUM = 1

session_dir = os.path.join(DATA_DIR, sessions[SESSION_NUM - 1])
games = sorted([f for f in os.listdir(session_dir) if f.startswith("game_") and f.endswith(".json")])
for g in games:
    with open(os.path.join(session_dir, g)) as f:
        data = json.load(f)
    w = data["winner"]
    label = f"{data['bot1']} wins" if w == "bot1" else f"{data['bot2']} wins" if w == "bot2" else "draw"
    time_info = f", {data['elapsed_seconds']}s" if 'elapsed_seconds' in data else ""
    print(f"  {g}: {label} ({data['move_count']} moves{time_info})")

## Load & Replay a Game
Set `GAME_PATH` to the game file you want to view, then run the cell to auto-play through the game.

In [ ]:
# Set the game file path here
GAME_NUM = 1
GAME_PATH = os.path.join(session_dir, games[GAME_NUM - 1])

# Delay between moves in seconds (for auto-play)
DELAY = 0.5

with open(GAME_PATH) as f:
    data = json.load(f)

bot1 = data["bot1"]
bot2 = data["bot2"]
bot1_color = data["bot1_color"]
white_bot = bot1 if bot1_color == "White" else bot2
black_bot = bot2 if bot1_color == "White" else bot1
moves = data["moves"]

w = data["winner"]
result_label = f"{bot1} wins" if w == "bot1" else f"{bot2} wins" if w == "bot2" else "Draw"

print(f"Game: {bot1} vs {bot2}")
print(f"White: {white_bot} | Black: {black_bot}")
print(f"Result: {data['result']} ({result_label}) | Moves: {data['move_count']} | Time: {data.get('elapsed_seconds', '?')}s")
if 'bot1_avg_move_time' in data:
    print(f"Avg move time: {bot1} {data['bot1_avg_move_time']}s | {bot2} {data['bot2_avg_move_time']}s")

In [ ]:
board = chess.Board()

for i, move_entry in enumerate(moves):
    # Support both enriched dicts and plain UCI strings
    if isinstance(move_entry, dict):
        move_uci = move_entry["uci"]
        move_time = move_entry.get("time")
    else:
        move_uci = move_entry
        move_time = None

    move = chess.Move.from_uci(move_uci)
    board.push(move)

    is_white_move = (i % 2 == 0)
    if is_white_move:
        mover = bot1 if bot1_color == "White" else bot2
    else:
        mover = bot1 if bot1_color == "Black" else bot2

    move_label = f"{mover}: {move_uci}"
    if move_time is not None:
        move_label += f" ({move_time}s)"

    clear_output(wait=True)
    print(f"Move {i + 1}/{len(moves)}  |  {move_label}")
    display(SVG(chess.svg.board(board, lastmove=move, size=400)))

    if i < len(moves) - 1:
        time.sleep(DELAY)

# Final result
print(f"\nResult: {data['result']} ({result_label})")